In [1]:
import datetime, time
import json
import urllib.parse
import pandas as pd
import configparser
from pprint import pprint
from tqdm import tqdm, tqdm_notebook
tqdm.pandas(tqdm_notebook)

import sqlite3

In [2]:
con = sqlite3.connect("data/pkp/main.db")
df = pd.read_sql_query("SELECT * from PKP", con)
df = df.set_index('doi')

In [3]:
def parse_json(row):
    if row['am_response']:
        row['am_response'] = json.loads(row['am_response'])

    if row['fb_og_object']:
        row['fb_og_object'] = json.loads(row['fb_og_object'])
        
    if row['fb_engagement']:
        row['fb_engagement'] = json.loads(row['fb_engagement'])
        
    return row
df = df.progress_apply(parse_json, axis=1)

278497it [00:59, 4649.97it/s]                            


In [4]:
def extract_values(row):
    row['fb_share_count'] = None
    row['fb_reaction_count'] = None
    row['fb_comment_plugin_count'] = None
    row['fb_comment_count'] = None
    row['fb_og_id'] = None
    row['am_id'] = None
    row['am_score'] = None
    row['am_share_count'] = None
    
    if row['fb_og_object']:
        row['fb_og_id'] = row['fb_og_object']['id']
    
    if row['fb_engagement']:
        row['fb_share_count'] = row['fb_engagement']['share_count']
        row['fb_reaction_count'] = row['fb_engagement']['reaction_count']
        row['fb_comment_plugin_count'] = row['fb_engagement']['comment_plugin_count']
        row['fb_comment_count'] = row['fb_engagement']['comment_count']
    
    if row['am_response']:
        try:
            row['am_id'] = row['am_response']['altmetric_id']
        except:
            pass

        try:
            row['am_score'] = row['am_response']['score']
        except:
            pass

        try:
            row['am_share_count'] = row['am_response']['counts']['facebook']['posts_count']
        except:
            pass
    
    return row
df = df.progress_apply(extract_values, axis=1)

278497it [23:38, 196.39it/s]                             


In [5]:
df.loc[df['doi_resolve_status'] == 200].describe()

,fb_share_count,fb_reaction_count,fb_comment_plugin_count,fb_comment_count,am_id,am_score,am_share_count
count,29575.000000,29575.000000,29575.0,29575.000000,1.365800e+04,14585.000000,5696.000000
mean,11.406661,261.541505,0.0,0.860321,9.862113e+06,3.137581,1.761587
std,66.928563,1695.941605,0.0,5.026204,8.228859e+06,16.305080,4.850970
min,0.000000,0.000000,0.0,0.000000,1.035680e+05,0.000000,1.000000
25%,0.000000,0.000000,0.0,0.000000,2.929668e+06,0.250000,1.000000
50%,0.000000,0.000000,0.0,0.000000,6.483838e+06,0.750000,1.000000
75%,1.000000,0.000000,0.0,0.000000,1.579229e+07,2.600000,2.000000
max,449.000000,11408.000000,0.0,210.000000,3.070235e+07,954.936000,276.000000


In [6]:
df[['timestamp',
    'doi_resolve_status', 'doi_resolve_error', 'doi_url', 'fb_response_error', 'am_response_error',
    'fb_og_id', 'fb_share_count', 'fb_reaction_count', 'fb_comment_plugin_count', 'fb_comment_count',
    'am_id', 'am_score', 'am_share_count']].to_csv("PKP_temp.csv")